<a href="https://colab.research.google.com/github/tronglinux123/AI-Project/blob/main/cxrretrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [ ]:
auth.authenticate_user()

In [ ]:
# Set up environment variables
project_id = 'project-physionet-474410'
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
dataset = 'mimiciv'


In [ ]:
data = pd.read_csv('processed_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'processed_data.csv'

In [ ]:
data.head()

In [ ]:
data2 = run_query(f"""
SELECT r.subject_id, dm.dicom, dm.StudyDate, h.hadm_id, h.admittime, h.dischtime
FROM `physionet-data.mimic_cxr.dicom_metadata_string` AS dm
JOIN `physionet-data.mimic_cxr.record_list` AS r ON r.dicom_id = dm.dicom
JOIN `physionet-data.mimiciv_2_2_hosp.admissions` AS h ON h.subject_id = r.subject_id
""")
print(data2)


/tmp/ipython-input-2544362523.py:7: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


         subject_id                                         dicom StudyDate  \
0          18415616  75d67482-46fbfcfb-b9d3be10-98f1b1dd-ba9748dc  21640428   
1          18415616  75d67482-46fbfcfb-b9d3be10-98f1b1dd-ba9748dc  21640428   
2          18415616  91ea24c1-ddf8f918-0c579885-c0bf36ed-3a2b306a  21640428   
3          18415616  91ea24c1-ddf8f918-0c579885-c0bf36ed-3a2b306a  21640428   
4          19136768  69ea47d2-8e44c7ea-8fd5dada-9385460a-fd8863d2  21481021   
...             ...                                           ...       ...   
2509603    17956863  4baead16-4c878e7c-fcc6071d-e06ebfb2-61d25c4c  22060814   
2509604    17956863  328fba1d-fb7a2246-f361ecc5-09c96079-48a89f06  22060927   
2509605    17956863  328fba1d-fb7a2246-f361ecc5-09c96079-48a89f06  22060927   
2509606    17956863  59218cf7-f2c291c7-25ed5f1f-1a839972-c93bb3b3  22060814   
2509607    17956863  59218cf7-f2c291c7-25ed5f1f-1a839972-c93bb3b3  22060814   

          hadm_id           admittime           dis

In [ ]:
data2['time'] = pd.to_datetime(data2['StudyDate'], format='%Y%m%d', errors='coerce').dt.strftime('%Y-%m-%dT%H:%M:%S')
data2['admittime'] = pd.to_datetime(data2['admittime']).dt.strftime('%Y-%m-%dT00:00:00')
data2

NameError: name 'pd' is not defined

In [ ]:
#Lọc Thời gian
data2 = data2[(data2['admittime'] <= data2['time']) & (data2['time'] <= data2['dischtime'])]

NameError: name 'data2' is not defined

In [ ]:
# Xóa các hàng 'hadm_id' trùng lặp, chỉ giữ lại lần xuất hiện đầu tiên
data2 = data2.drop_duplicates(subset='dicom', keep='first')

print(data2)

         subject_id                                         dicom StudyDate  \
0          18415616  75d67482-46fbfcfb-b9d3be10-98f1b1dd-ba9748dc  21640428   
2          18415616  91ea24c1-ddf8f918-0c579885-c0bf36ed-3a2b306a  21640428   
13         19136768  c2822cd9-785880dd-b21df2f4-feae6873-a3dbcc34  21501224   
27         19136768  35469b32-ecaab79e-6029f852-160bd61a-9e39d5bb  21510408   
33         19136768  aad41f8e-432a7f56-264d490c-740a1fa6-94f4f537  21510408   
...             ...                                           ...       ...   
2509593    15007487  81f647b5-856ae069-e0941845-c6ddb0f6-e9a9aaec  21421214   
2509594    15007487  2acfc474-14dcc92e-4c225ec7-4451198e-601be5b0  21421213   
2509595    15007487  4690a275-0ffc1451-af9921ce-cebf8df2-24a0236a  21421215   
2509596    17956863  aaf308c6-4bdb0059-affc2594-f26cb1f6-702c5e9d  22060927   
2509604    17956863  328fba1d-fb7a2246-f361ecc5-09c96079-48a89f06  22060927   

          hadm_id            admittime           di

In [ ]:
# Merge the two DataFrames on 'dicom'
data_f = data.merge(data2, on='dicom', how='inner')
print(data_f)

NameError: name 'data' is not defined

In [ ]:
# Drop duplicate 'hadm_id' rows, keeping only the first occurrence
data_f = data_f.drop_duplicates(subset='hadm_id', keep='first')

NameError: name 'data_f' is not defined

In [ ]:
data_f.to_csv('x_ray_data.csv', index=False)